In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import json
import sys
from pathlib import Path
import re 

import blpapi
import ipywidgets as w
import ipyvuetify as v
import numpy as np
import pandas as pd
import QuantLib as ql
from IPython.display import display
from ipydatagrid import DataGrid

root_path = Path('/dev/projects/cds/')

if (path := str(root_path.resolve())) not in sys.path:
    sys.path.append(path)

from api.blpw import BlpQuery
from api.data.base import data_path
from api.data.curves import curves
from api.data.utils import add_pcs, parse_tenor
from api.pricing.cds import get_isda_cds
from api.data.processing import apply_function
from api.pricing.utils import (
    get_cds_specs,
    get_cds_curve,
    get_cds_spreads,
    get_risk_free_curve,
    get_cds_implied_price,
    get_curve,
)

In [2]:
# Technically, the model requires the discount factor to be 1 at spot;
# but we can't do that and also have the discount curve extend back to
# the trade date.  For the time being, we'll keep discount = 1 at trade.
# The results match anyway.

In [3]:
securities = [
    'XS1968706876 Corp',
    'XS2109813142 Corp',
    'US195325CX13 Corp',
    'US195325DX04 Corp',
    'XS1558078736 Corp',
    'XS2391398174 Corp',
    'XS1843443356 Corp',
    'US900123CM05 Corp',
    'CMEX1U5 Corp',
    'XS2289588167 Corp',
    'CCHIL1U5 Corp',
    'US168863DU93 Corp',
    'US718286CB15 Corp',
    'US718286CJ41 Corp',
    'US715638BM30 Corp',
    'US715638DQ26 Corp',
    'XS2050933626 Corp',
    'XS1263139856 Corp',
    'XS1245432742 Corp',
    'XS2330514899 Corp',
    'XS2178857285 Corp',
    'XS1694217495 Corp',
    'XS1936302949 Corp',
    'XS1807174393 Corp',
    'XS1959337749 Corp',
    'US195325DP79 Corp',
    'USM88269US88 Corp',
    'XS2445169985 Corp',
    'US836205BA15 Corp',
    'US836205AV60 Corp',
    'US900123AL40 Corp',
    'XS2455985569 Corp',
    '91282CEC1 Corp',
    'XS2455984679 Corp',
    '91282CDY4 Corp',
    'US715638DP43 Corp',
    'US718286CQ83 Corp',
    'US718286CR66 Corp',
    'USY6972HLP91 Corp',
    'US760942BA98 Corp',
    'US760942BB71 Corp',
    'US455780DJ24 Corp',
    'XS1120709826 Corp',
    'CTURK1U3 Corp',
    'US105756CB40 Corp',
    'CBRZ1U5 Corp',
    'US445545AF36 Corp',
    'XS2388586583 Corp',
    'XS1775617464 Corp',
    'US455780CS32 Corp',
    'US455780CT15 Corp',
    'XS2364200514 Corp',
    'XS1313004928 Corp',
    'US715638DF60 Corp',
    'CEGY1U5 Corp',
    'US91087BAL45 Corp',
    'CCOL1U5 Corp',
    'US040114HT09 Corp',
    'US040114HV54 Corp',
    'XS2468421248 Corp',
    'US900123CL22 Corp',
    'US91087BAN01 Corp',
    'US91087BAS97 Corp',
    'XS2159975700 Corp',
    'XS2159975882 Corp',
    'XS2225210330 Corp',
    'XS1696899035 Corp',
    'XS2446175577 Corp',
    'USY7141BAA18 Corp',
    'USY7141BAB90 Corp',
    'XS2259191430 Corp',
    'CPERU1U5 Corp',
    'CSOAF1U5 Corp',
    'US836205BC70 Corp',
    'US836205BE37 Corp',
    'BV8695652 Corp',
    'US195325EA91 Corp',
    'CTURK1U5 Corp',
    'US836205BB97 Corp',
    'XS1892141620 Corp',
    'US698299BL70 Corp',
    'US698299AW45 Corp',
    'XS2109766472 Corp',
    'US455780CW44 Corp',
    'US455780DG84 Corp',
    'XS2270577344 Corp',
    'XS2270576619 Corp',
    'XS2434895806 Corp',
    'XS1028952403 Corp',
    'XS1781710626 Corp',
    'XS1843435840 Corp',
    'USP3579ECB13 Corp',
    'USP3579ECG00 Corp',
    'USP3699PGH49 Corp',
    'XS1108847531 Corp',
    'XS2325747637 Corp',
    'XS1236685613 Corp',
    'XS2388562139 Corp',
    'USP75744AK10 Corp',
    'USP75744AB11 Corp',
    'XS2472852610 Corp',
    'XS1382696398 Corp',
    'XS2079846635 Corp',
    'US718286BG11 Corp',
    'US718286CL96 Corp',
    'XS1968714623 Corp',
    'US836205AY00 Corp',
    'CTUR1U10 Corp',
    'CY002690 Corp',
    'XS2434896010 Corp',
    'XS2447602793 Corp',
    'XS2485248806 Corp',
    'XS2291692890 Corp',
    'XS2388586401 Corp',
    'US71567RAV87 Corp',
    'XS2487342649 Corp',
    'US900123CB40 Corp',
    'XS1057340009 Corp',
    'US900123CA66 Corp',
    'US41809JAA34 Corp',
    'XS2010026305 Corp',
    'XS2010026487 Corp',
    'XS2485249523 Corp',
    'US105756BX78 Corp',
    'CTURK1U1 Corp',
    'US195325BQ70 Corp',
    'XS2280637039 Corp',
    'US900123BJ84 Corp',
    'XS1717013095 Corp',
    'XS1312891549 Corp',
    'US900123CF53 Corp',
    'XS1821416234 Corp',
    'XS2258400162 Corp',
    'USP7S08VCA70 Corp',
    'US71654QBX97 Corp',
    'XS2210006339 Corp',
    'XS2386638733 Corp',
    'XS2391394348 Corp',
    'XS1837994794 Corp',
    'XS1369323149 Corp',
    'US31424EAD40 Corp',
    'US836205AX27 Corp',
    'XS1843435766 Corp',
    'XS2176899701 Corp',
    'XS0505478684 Corp',
    'XS1106137687 Corp',
    'XS2391395154 Corp',
    'XS2451768720 Corp',
    'XS2297221405 Corp',
]

#### Functions

#### Settings

In [4]:
dt_fmt = '%Y-%m-%d'
bbg_dt_fmt = '%Y%m%d'

#### Bloomberg Connection

In [5]:
bq = BlpQuery(timeout=50000).start()
self = bq

#### Common

In [6]:
ref_fields = [
    'security_des', 
    'security_typ'
]

In [7]:
ref = bq.bdp(
    securities=securities,
    fields=ref_fields
)

#### Bonds

In [8]:
bond_mkt_fields = [
    'yas_bond_px',
    'yas_zspread',
    'yas_mod_dur'
]

In [9]:
bond_ref = ref.loc[~(ref['security_typ'] == 'CREDIT DEFAULT SWAP'), :]

In [10]:
bond_pcs = 'BGN'
bond_sep = '@'

bond_mkt = bq.bdp(
    securities=bond_ref['security'].to_list() if not bond_pcs else add_pcs(bond_ref['security'], source=bond_pcs, sep=bond_sep),
    fields=bond_mkt_fields
)
if bond_pcs:
    bond_mkt['security'] = bond_mkt['security'].str.replace(f'{bond_sep}{bond_pcs}', '')

In [12]:
def get_overrides(scenarios, data):
    parsed = []
    for scen in scenarios:
        override_fields = ensure_list(scen['override_fields'])
        column_names = scen.get('override_values')
        column_names = ensure_list(column_names) if column_names else override_fields
        ovrds = {k: v for k, v in zip(override_fields, ensure_list(scen['fn'](*data[column_names])))}
        parsed.append(BBGOverrides(label=scen['name'], **ovrds))
    
    # return [s['fn'](row[key]) for s in scen['scenarios']]
    return parsed 

In [13]:
class BBGList:
    def __init__(self, elems):
        self.elems = elems
              
class BBGRequestParams:
    def __init__(
        self,
        securities,
        fields,
        overrides=None,
    ):
        self.securities = securities
        self.fields = fields
        self.overrides = overrides
        
        
class BBGSecurity:
    def __init__(
        self,
        id,
        id_type='ticker',
        overrides=None,
        pcs=None,
        sep=' ',
    ):
        self.id = id
        self.id_type = id_type
        self.pcs = pcs
        self.sep = sep
        self.overrides = overrides

        
class BBGField:
    def __init__(
        self,
        field,
        name=None,
        overrides=None
    ):
        self.field = field
        self.name = name or field
        self.overrides = overrides
        
        
class BBGOverrides:
    def __init__(
        self,
        label=None,
        **kwargs
    ):
        self.label = label
        self.kwargs = kwargs

In [14]:
def ensure_list(obj):
    if not isinstance(obj, list):
        return [obj]
    return obj

In [15]:
bonds_scenarios = [
    # {
    #     'override_fields': ['yas_zspread', 'yas_mod_dur'],
    #     'name': 'double z',
    #     'fn': lambda x, y: [x * 2, y / 2]
    # },
    {
        'override_fields': 'yas_zspread',
        'override_values': 'yas_zspread',
        'name': 'double z',
        'fn': lambda x: x * 2
    },
    {
        'override_fields': 'yas_zspread',
        'override_values': 'yas_zspread',
        'name': 'half z',
        'fn': lambda x: x / 2
    },
]

Case1:
securities = list of str,
fields = list of str 
overrides 


case overrides specific to securities fields in common
case overrides specific to fields 

In [16]:
bbg_securities = [
    BBGSecurity(
        id=row['security'],
        overrides=get_overrides(scenarios=bonds_scenarios, data=row)
    ) for index, row in bond_mkt.dropna().iterrows()
][:]

bbg_fields = [BBGField(f) for f in ['yas_bond_px', 'yas_mod_dur']]

In [17]:
bbg_request_params = BBGRequestParams(
    securities=bbg_securities,
    fields=bbg_fields
)
request_params = bbg_request_params

In [30]:
bbg_securities[2].overrides

In [18]:
%%time
data = self.bdpx(request_params)

CPU times: total: 297 ms
Wall time: 13.8 s


In [19]:
data

security  yas_bond_px  yas_mod_dur  yas_zspread security_label
0    XS1968706876 Corp   101.599215    14.901195   227.341426         half z
1    XS1968706876 Corp    43.751902     9.027412   909.365702       double z
2    XS2109813142 Corp    83.900652    16.215708   220.066707         half z
3    XS2109813142 Corp    34.276543     9.744818   880.266827       double z
4    US195325CX13 Corp    98.478686     3.054357   127.693349         half z
..                 ...          ...          ...          ...            ...
269  XS2391395154 Corp    39.852712     4.668908  1863.045564       double z
270  XS2451768720 Corp    87.706938     4.370003   618.854294       double z
271  XS2451768720 Corp   108.305708     4.582813   154.713573         half z
272  XS2297221405 Corp    97.229106    12.194179   472.900208         half z
273  XS2297221405 Corp    33.618314     4.414877  1891.600833       double z

[274 rows x 5 columns]

In [25]:
data.pivot(index='security', columns='security_label').reorder_levels([1, 0], axis=1).sort_index(level=0, axis=1)

security_label       double z                               half z  \
                  yas_bond_px yas_mod_dur  yas_zspread yas_bond_px   
security                                                             
91282CDY4 Corp      90.061755    8.586953    38.528708   92.346973   
91282CEC1 Corp      94.662312    4.375249    47.041671   96.145876   
BV8695652 Corp      85.773696    8.127347   257.639853  100.691244   
US040114HT09 Corp    8.180417    3.341523  4341.822604   46.481907   
US040114HV54 Corp   11.946907    3.438595  3627.801158   50.908191   
...                       ...         ...          ...         ...   
XS2468421248 Corp   64.193142    3.095365  2096.928895  111.090167   
XS2472852610 Corp   92.398272    2.526502   607.695498  103.949526   
XS2485248806 Corp   81.825500    4.434153   699.743684  104.012301   
XS2485249523 Corp   65.570466    7.298237   860.851047  109.902809   
XS2487342649 Corp   89.475484    8.661132   122.961191   96.975690   

security_label                              
                  yas_mod_dur  yas_zspread  
security                                    
91282CDY4 Corp       8.613554     9.632177  
91282CEC1 Corp       4.385016    11.760418  
BV8695652 Corp       8.396683    64.409963  
US040114HT09 Corp    7.134839  1085.455651  
US040114HV54 Corp    6.528962   906.950289  
...                       ...          ...  
XS2468421248 Corp    3.677499   524.232224  
XS2472852610 Corp    2.597496   151.923874  
XS2485248806 Corp    4.647252   174.935921  
XS2485249523 Corp    8.571764   215.212762  
XS2487342649 Corp    8.783793    30.740298  

[136 rows x 6 columns]

In [ ]:
while self.session.tryNextEvent():
    pass
operation = 'ReferenceDataRequest'

sent_events = 0
service = self._services[self._SERVICES[operation]]

bbg_secs = request_params.securities
bbg_fields = request_params.fields
if hasattr(bbg_secs, BBGList):  # assuming overrides specific to securities and fields in common
    secs = []
    for j, sec in enumerate(bbg_secs.elems):

        security = sec.id
        secs.append(security)
        
        for overrides in sec.overrides:
            request = service.createRequest(operation)
            request.getElement('securities').appendValue(security)
            bbg_overrides = request.getElement('overrides')
            for f in bbg_fields:
                request.getElement('fields').appendValue(f)

            for k, v in overrides.kwargs.items():
                ovrd = bbg_overrides.appendElement()
                ovrd.setElement('fieldId', k)
                ovrd.setElement('value', v)

            cid_kwargs = overrides.kwargs if overrides.label else {'_label': overrides.label, **overrides.kwargs}
            cid = blpapi.CorrelationId(json.dumps(overrides.kwargs))
            # logger.info('Sending Request:\n{}'.format(request))
            self.session.sendRequest(request, identity=self.identity, correlationId=cid)
            sent_events += 1
elif isinstance(bbg_fields, BBGList):
    for 


In [ ]:
sec.overrides[0].kwargs

In [ ]:
while self.session.tryNextEvent():
    pass
operation = 'ReferenceDataRequest'

sent_events = 0
service = self._services[self._SERVICES[operation]]

bbg_securities = request_params.securities
bbg_fields = request_params.fields
secs = []
use_cid = False
for j, sec in enumerate(bbg_securities):
    security = sec.id
    secs.append(security)
    for field in bbg_fields:
        if sec.overrides:
            use_cid = True
            for sec_overrides in sec.overrides:
                request = service.createRequest(operation)
                request.getElement('securities').appendValue(security)
                bbg_overrides = request.getElement('overrides')
                request.getElement('fields').appendValue(field.field)

                for k, v in sec_overrides.kwargs.items():
                    ovrd = bbg_overrides.appendElement()
                    ovrd.setElement('fieldId', k)
                    ovrd.setElement('value', v)
                
                cid_kwargs = {**sec_overrides.kwargs} 
                if sec_overrides.label: 
                    cid_kwargs['security_label'] = sec_overrides.label
                
                if field.overrides:

                    for k, v in field.overrides.kwargs.items():
                        ovrd = bbg_overrides.appendElement()
                        ovrd.setElement('fieldId', k)
                        ovrd.setElement('value', v)
                    
                    cid_kwargs.update(field.overrides.kwargs)
                    if field.overrides.label: 
                        cid_kwargs['field_label'] = field.overrides.label
            
                cid = blpapi.CorrelationId(json.dumps(cid_kwargs))
                # logger.info('Sending Request:\n{}'.format(request))
                self.session.sendRequest(request, identity=self.identity, correlationId=cid)
                sent_events += 1
        else:
            request = service.createRequest(operation)
            request.getElement('securities').appendValue(security)
            bbg_overrides = request.getElement('overrides')
            request.getElement('fields').appendValue(field.field)

            if field.overrides:
                use_cid = True
                for k, v in field.overrides.kwargs.items():
                    ovrd = bbg_overrides.appendElement()
                    ovrd.setElement('fieldId', k)
                    ovrd.setElement('value', v)

                cid_kwargs = {**field.overrides.kwargs}
                if field.overrides.label: 
                    cid_kwargs['field_label'] = field.overrides.label

                cid = blpapi.CorrelationId(cid_kwargs)
            else:
                cid = blpapi.CorrelationId()
            # logger.info('Sending Request:\n{}'.format(request))
            self.session.sendRequest(request, identity=self.identity, correlationId=cid)
            sent_events += 1
            
data = []
for msg in msgs:
    sec_data = msg['element']['ReferenceDataResponse'][0]['securityData']
    d = {
        'security': sec_data['security'],
        **sec_data['fieldData']['fieldData']
    }
    
    drop_duplicates = False
    if use_cid:
        grouper = json.loads(msg['correlationIds'][0])
        d.update(grouper)
        drop_duplicates = True
        
    data.append(d)

data = pd.DataFrame(data).set_index('security').loc[secs, :].reset_index()

if drop_duplicates:
    data = data.groupby([*grouper.keys()], sort=False, as_index=False).apply(lambda g: g.ffill().bfill().drop_duplicates()).reset_index(drop=True)

In [ ]:
%%time
msgs = [msg for msg in self._receive_events(sent_events)]

In [ ]:
data = []
for msg in msgs:
    sec_data = msg['element']['ReferenceDataResponse'][0]['securityData']
    d = {
        'security': sec_data['security'],
        **sec_data['fieldData']['fieldData']
    }
    
    drop_duplicates = False
    if use_cid:
        grouper = json.loads(msg['correlationIds'][0])
        d.update(grouper)
        drop_duplicates = True
        
    data.append(d)

data = pd.DataFrame(data).set_index('security').loc[secs, :].reset_index()

if drop_duplicates:
    data = data.groupby([*grouper.keys()], sort=False, as_index=False).apply(lambda g: g.ffill().bfill().drop_duplicates()).reset_index(drop=True)

In [ ]:
data

In [ ]:
dd.ffill().bfill().drop_duplicates()

In [ ]:
data = []
for msg in msgs:
    id_elems = msg['correlationIds'][0].split('__')
    sec_data = msg['element']['ReferenceDataResponse'][0]['securityData']

    labels = {'label': id_elems[2]} if len(id_elems) == 3 else {}
    d = {
        'security': sec_data['security'],
        **labels,
        id_elems[0]: id_elems[1],
        **sec_data['fieldData']['fieldData']
    }
    data.append(d)

data = pd.DataFrame(data).set_index('security').loc[secs, :].reset_index()

In [ ]:
overrides

override specific to security 
override specific to fied 

In [ ]:
args

In [ ]:
x = {
    'a': [{'a': 1}, {'b': 4}],
    'b': [5, 7]
}

In [ ]:
[*zip(*x.values())]

In [ ]:
data = self.bdpm(args[:10])

In [ ]:
data.pivot(index='security', columns='label').reorder_levels([1, 0], axis=1).sort_index(level=0, axis=1)

In [ ]:
bond_base = pd.concat([bond_mkt.set_index('security')], keys=['base'], axis=1).rename_axis(['scenario', None], axis=1)
bond_stress = pd.concat(bond_res).reset_index().pivot(index='security', columns='scenario').reorder_levels([1, 0], axis=1).sort_index(level=0, axis=1)

In [ ]:
bond_df = pd.concat(
    [
        bond_base,
        bond_stress,
    ], axis=1
).dropna()

In [ ]:
bond_df = bond_df.set_index(bond_df.index.map(bond_ref.set_index('security')['security_des']))

In [ ]:
dg = DataGrid(df)
dg.auto_fit_columns = True
dg

#### CDS

In [ ]:
cds_mkt_fields = [
    # 'cds_quoted_price',
    'px_last',
    'upfront_last',
]

In [ ]:
cds_ref = ref.loc[ref['security_typ'] == 'CREDIT DEFAULT SWAP', :]

In [ ]:
cds_pcs = 'MSG1'
cds_sep = ' '

cds_mkt = bq.bdp(
    securities=cds_ref['security'].to_list() if not cds_pcs else add_pcs(cds_ref['security'], source=cds_pcs, sep=cds_sep),
    fields=cds_mkt_fields
)

if cds_pcs:
    cds_mkt['security'] = cds_mkt['security'].str.replace(f'{cds_sep}{cds_pcs}', '')

In [ ]:
cds_spread_scenarios = {
    'override_field': 'cds_flat_spread',
    'override_value': 'px_last',
    'scenarios': [
        {
            'name': 'double z',
            'fn': lambda x: x * 2
        },
        {
            'name': 'half z',
            'fn': lambda x: x / 2
        },
    ]
} 

In [ ]:
n = 5
cds_res = []
for index, row in cds_mkt.iterrows():
    if index + 1 <= n:
        cds_res.append(get_stressed(row, ['upfront_last'], cds_spread_scenarios))

In [ ]:
cds_base = pd.concat([cds_mkt.set_index('security')], keys=['base'], axis=1).rename_axis(['scenario', None], axis=1)
cds_stress = pd.concat(cds_res).reset_index().pivot(index='security', columns='scenario').reorder_levels([1, 0], axis=1).sort_index(level=0, axis=1)

In [ ]:
cds_df = pd.concat(
    [
        cds_base,
        cds_stress,
    ], axis=1
).dropna()
cds_df = cds_df.set_index(cds_df.index.map(cds_ref.set_index('security')['security_des']))

In [ ]:
cds_df